# Laboratorio 4: N-gramas

**Universidad del Valle de Guatemala**  
**Facultad de Ingeniería**  
**Departamento de Ciencias de la Computación**  
**Procesamiento de Lenguaje Natural**

## Integrantes

- Diego Leiva       -   21752
- Maria Ramirez     -   21342
- Pablo Orellana    -   21970

## Implementación de Modelo

### Librerías

In [ ]:
import re
from collections import Counter
import math
import matplotlib.pyplot as plt

### Lectura del corpus

In [ ]:
corpus = """
Cristiano Ronaldo forma parte de la leyenda del Real Madrid y siempre será recordado como uno de los grandes símbolos de su historia. 
Fue presentado en el Santiago Bernabéu el 6 de julio de 2009, acompañado por Eusebio y Alfredo Di Stéfano, y desde entonces no paró 
de marcar goles: 451 en 438 partidos oficiales con el Real Madrid (más de uno por encuentro). Además vio puerta en todas las 
competiciones: 312 en la Liga, 105 en la Champions, 22 en la Copa del Rey, 6 en el Mundial de Clubes, 4 en la Supercopa de España 
y 2 en la Supercopa de Europa.

Nadie en la historia del club ha marcado tantos goles como el delantero portugués, cuyo palmarés en el Real Madrid es impresionante: 
4 Champions League, 3 Mundiales de Clubes, 3 Supercopas de Europa, 2 Ligas, 2 Copas del Rey y 2 Supercopas de España. A ellos suma 
4 Balones de Oro, 3 Botas de Oro, 2 Premios The Best, 3 Premios al Mejor Jugador de la UEFA y 3 Trofeos Pichichi.

Trayectoria de récords
A lo largo de sus nueve temporadas como madridista, Cristiano dejó varios récords impresionantes: máximo goleador de la historia 
del club, máximo goleador de la historia de la Copa de Europa (marcó 105 tantos en Champions con nuestro equipo); máximo goleador 
madridista en la Liga (312); más partidos de tres o más goles en la historia de la Liga (34); y más tantos en una temporada con el 
Real Madrid (61).
"""

print("Cantidad de párrafos:", corpus.count('\n\n') + 1)

### Preprocesamiento y Tokenización

In [ ]:
# Limpieza del texto
texto_limpio = re.sub(r'[^\w\s]', '', corpus.lower())

# Tokenización simple
tokens = texto_limpio.split()

# Creacion de Unigramas y Bigrama
unigrams = tokens
bigrams = list(zip(tokens, tokens[1:]))

# Conteo de bigramas y unigramas
unigram_counts = Counter(tokens)
bigram_counts = Counter(bigrams)
vocabulario = set(tokens)

# Mostrar resultados
print(f"Conteos")
print(f"Unigramas: {len(unigram_counts)}")
print(f"Bigramas: {len(bigram_counts)}")
print("-" * 20)
print("Vocabulario")
print(f"Primeros 10 unigramas: {unigram_counts[:10]}")  # Muestra los 10 primeros unigramas
print(f"Primeros 10 bigramas: {bigram_counts[:10]}")  # Muestra los 10 primeros bigramas
print(f"Tamaño del vocabulario: {len(vocabulario)}")
print(f"Primeros 10 palabras del vocabulario: {list(vocabulario)[:10]}")

### Visualización de Frecuencias

In [ ]:
# Obtener las 10 n-gramas más frecuentes
top_unigrams = unigram_counts.most_common(10)
top_bigrams = bigram_counts.most_common(10)

# Visualización de las frecuencias
def plot_frequencies(frequencies, title):
    """
    Genera un gráfico de barras para las frecuencias de n-gramas.
    
    Args:
        frequencies (list of tuples): Lista de tuplas con n-gramas y sus frecuencias.
        title (str): Título del gráfico.

    Returns:
        None
    """
    labels, values = zip(*frequencies)
    plt.figure(figsize=(10, 5))
    plt.bar(labels, values, color='blue')
    plt.title(title)
    plt.xlabel('N-gramas')
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_frequencies(top_unigrams, 'Top 10 Unigramas')

In [ ]:
plot_frequencies(top_bigrams, 'Top 10 Bigramas')

#### Cálculo de Entropía y Preplejidad sin Smoothing

In [ ]:
frase = "cristiano ronaldo forma parte de la leyenda"
# Dividir la frase en tokens y calcular bigramas
tokens_frase = frase.lower().split()
bigrams_frase = list(zip(tokens_frase[:-1], tokens_frase[1:]))

probabilidades = [] # Lista para almacenar las probabilidades de los bigramas

# Calcular probabilidades de los bigramas en la frase
for bigrama in bigrams_frase:
    # Obtener la frecuencia del bigrama y del unigrama
    c_bigram = bigram_counts.get(bigrama, 0)
    c_unigram = unigram_counts.get(bigrama[0], 1)
    # Calcular la probabilidad del bigrama
    prob = c_bigram / c_unigram
    # Agregar la probabilidad a la lista
    probabilidades.append(prob)

# Calcular entropía y perplejidad
entropia = -sum(math.log2(p) for p in probabilidades if p > 0) / len(probabilidades)
perplejidad = 2 ** entropia

# resultados
print("Frase:", frase)
print("Bigrama de la frase:", bigrams_frase)
print("Probabilidades:", probabilidades)
print("Entropía (sin smoothing):", round(entropia, 4))
print("Perplejidad (sin smoothing):", round(perplejidad, 4))

#### Técnicas de Smoothing Laplace y Add-K

In [ ]:
V = len(vocabulario)  # Tamaño del vocabulario
k = 0.5    # Valor de k para Add-K smoothing

# Inicializar listas para almacenar las probabilidades con smoothing
prob_laplace = []
prob_add_k = []

# Calcular probabilidades con Laplace y Add-K smoothing
# para cada bigrama en la frase
for bigrama in bigrams_frase:
    # Obtener la frecuencia del bigrama y del unigrama
    c_bigram = bigram_counts.get(bigrama, 0)
    c_unigram = unigram_counts.get(bigrama[0], 0)

    # Calcular las probabilidades con Laplace y Add-K smoothing
    # Laplace smoothing: P(w2|w1) = (C(w1, w2) + 1) / (C(w1) + V)
    # Add-K smoothing: P(w2|w1) = (C(w1, w2) + k) / (C(w1) + k * V)
    laplace = (c_bigram + 1) / (c_unigram + V)
    addk = (c_bigram + k) / (c_unigram + k * V)

    # Agregar las probabilidades a las listas
    prob_laplace.append(laplace)
    prob_add_k.append(addk)


# Entropía y Perplejidad Laplace
entropia_laplace = -sum(math.log2(p) for p in prob_laplace) / len(prob_laplace)
perplejidad_laplace = 2 ** entropia_laplace

# Entropía y Perplejidad Add-K
entropia_addk = -sum(math.log2(p) for p in prob_add_k) / len(prob_add_k)
perplejidad_addk = 2 ** entropia_addk

print("Entropía Laplace:", round(entropia_laplace, 4))
print("Perplejidad Laplace:", round(perplejidad_laplace, 4))
print("Entropía Add-K:", round(entropia_addk, 4))
print("Perplejidad Add-K:", round(perplejidad_addk, 4))